# Assignment-7 📚
## CSE-467 - Parallel and Distributed Computing 💻
### Syed Owais Ali - 23053

Design EREW-PRAM model for linear regression for 16 elements and then implement it in CUDA-C or CUDA-Python for N elements. Compare the sequential implementation with parallel implementation. Fill execution time in the following table for the given values of N. You can implement the solution with local shared variable among threads within a block and with global variable shared among all blocks. 

---
  

|N (Array length) | Sequential | Shared variable Partial Sums (GPU)| Global variable One Final Sum (GPU) |
|--|--| -- | -- |
|  1 x 10<sup>6</sup> |  | | |
|  10 x 10<sup>6</sup>|  | | |
|  20 x 10<sup>6</sup>|  | | |
|  35 x 10<sup>6</sup>|  | | |
|  40 x 10<sup>6</sup>|  | | |
|  50 x 10<sup>6</sup>|  | | |





## Setup

In [1]:
!nvidia-smi

Mon May 15 20:45:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.5 LTS
Release:	20.04
Codename:	focal


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-so3iq5r1
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-so3iq5r1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=6dfacd6feb0ae3b3236bacc98df11e538bfd19392e14ca6e1593528ac6682eca
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ebw3p8a/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Sequential

In [90]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <math.h>
#define N 1000000

int main()
{
    // Number of bytes to allocate for N doubles
    size_t bytes = N*sizeof(int);

    // Allocate memory for arrays A, B on host
    int *X = (int*)malloc(bytes);
    int *Y = (int*)malloc(bytes);
    long double sum_y = 0.0;
    long double sum_x = 0.0;
    long double sum_x_squared = 0.0;
    long double sum_xy = 0.0;
    long double A = 0.0;
    long double B = 0.0;
  
    struct timeval start, end;

  
    for (int i = 0; i < N; i++) {
        X[i] = rand() % 100;
        Y[i] = rand() % 100;
    }

    gettimeofday(&start, NULL);

    
    for (int i = 0; i < N; i++) {
        sum_y += Y[i];
        sum_x += X[i];
        sum_x_squared += pow(X[i], 2);
        sum_xy += (X[i] * Y[i]);
    }
 
    // printf("sum_y %d\n", sum_y);
    // printf("sum_x %d\n", sum_x);
    // printf("sum_x_squared %llu\n", sum_x_squared);
    // printf("sum_xy %llu\n", sum_xy);
 
    A = ((sum_y * sum_x_squared) - (sum_x * sum_xy))/((N * sum_x_squared) - pow(sum_x, 2));
    B = ((N * sum_xy) - (sum_x * sum_y))/((N * sum_x_squared) - (sum_x_squared));
 
    gettimeofday(&end, NULL);
    long seconds = (end.tv_sec - start.tv_sec);
    long micros = ((seconds * 1000000) + end.tv_usec) - (start.tv_usec);
 
    printf("y= a + bx\n");
    printf("Sequential Linear Regression: A = %Lf and B = %Lf\n", A, B);
    printf("The elapsed time by CPU is %ds and %dus \n", seconds, micros); 

    free(X);
    free(Y);

    return 0;
}

y= a + bx
Sequential Linear Regression: A = 49.509458 and B = -0.000154
The elapsed time by CPU is 0s and 35001us 



## Shared variable Partial Sums (GPU)

In [103]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <math.h>
#define N 50000000


__global__ void linear_regression_shared(double *A, double *B, int *X, int *Y, int *sum_y, int *sum_x, int *sum_x_squared, int *sum_xy)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    int tid = threadIdx.x;
    int blockId = blockIdx.x;
    __shared__ int temp_y[256];
    __shared__ int temp_x[256];
    __shared__ int temp_x_sqr[256];
    __shared__ int temp_xy[256];
 
    if(id < N) {
        temp_y[tid] =  Y[id];
        temp_x[tid] =  X[id];
        temp_x_sqr[tid] =  X[id] * X[id];
        temp_xy[tid] =  X[id] * Y[id];
    }
 
    __syncthreads();
 
    if (tid == 0) {
        for(int i =0; i<256; i++){
            atomicAdd(sum_y, temp_y[i]);
            atomicAdd(sum_x, temp_x[i]);
            atomicAdd(sum_x_squared, temp_x_sqr[i]);
            atomicAdd(sum_xy, temp_xy[i]);
        }
    }
 
    if (id == 0) {
        *A = ((double) (*sum_y * *sum_x_squared) - (*sum_x * *sum_xy))/((N * *sum_x_squared) - pow(*sum_x, 2));
        *B = ((double) (N * *sum_xy) - (*sum_x * *sum_y))/((N * *sum_x_squared) - (*sum_x_squared));
    }

}

int main()
{
    // Number of bytes to allocate for N doubles
    size_t bytes = N*sizeof(int);

    // Allocate memory for arrays A, B on host
    int *X = (int*)malloc(bytes);
    int *Y = (int*)malloc(bytes);
    long double sum_y = 0.0;
    long double sum_x = 0.0;
    long double sum_x_squared = 0.0;
    long double sum_xy = 0.0;
    long double A = 0.0;
    long double B = 0.0;
  
    struct timeval start, end;

  
    for (int i = 0; i < N; i++) {
        X[i] = rand() % 10;
        Y[i] = rand() % 10;
    }

    gettimeofday(&start, NULL);

    
    for (int i = 0; i < N; i++) {
        sum_y += Y[i];
        sum_x += X[i];
        sum_x_squared += pow(X[i], 2);
        sum_xy += (X[i] * Y[i]);
    }
 
    // printf("sum_y %d\n", sum_y);
    // printf("sum_x %d\n", sum_x);
    // printf("sum_x_squared %llu\n", sum_x_squared);
    // printf("sum_xy %llu\n", sum_xy);
 
    A = ((sum_y * sum_x_squared) - (sum_x * sum_xy))/((N * sum_x_squared) - pow(sum_x, 2));
    B = ((N * sum_xy) - (sum_x * sum_y))/((N * sum_x_squared) - (sum_x_squared));
 
    gettimeofday(&end, NULL);
    long seconds = (end.tv_sec - start.tv_sec);
    long micros = ((seconds * 1000000) + end.tv_usec) - (start.tv_usec);
 
    printf("y= a + bx\n");
    printf("Sequential Linear Regression: A = %Lf and B = %Lf\n", A, B);
    printf("The elapsed time by CPU is %ds and %dus \n", seconds, micros); 
 
    int *d_X, *d_Y;
    double *d_A, *d_B;
    int *d_sum_y, *d_sum_x, *d_sum_x_squared, *d_sum_xy;
    cudaMalloc(&d_X, bytes);
    cudaMalloc(&d_Y, bytes);
    cudaMalloc(&d_A, sizeof(double));
    cudaMalloc(&d_B, sizeof(double));
    cudaMalloc(&d_sum_y, sizeof(int));
    cudaMalloc(&d_sum_x, sizeof(int));
    cudaMalloc(&d_sum_x_squared, sizeof(int)); 
    cudaMalloc(&d_sum_xy, sizeof(int)); 
 
    cudaMemcpy(d_X, X, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_Y, Y, bytes, cudaMemcpyHostToDevice);
 
    int thr_per_blk = 256;
    int blk_in_grid = ceil( float(N) / thr_per_blk );

    // Launch kernel
    gettimeofday(&start, NULL);
    linear_regression_shared<<< blk_in_grid, thr_per_blk >>>(d_A, d_B, d_X, d_Y, d_sum_y, d_sum_x, d_sum_x_squared, d_sum_xy);
 
    cudaDeviceSynchronize();
 
    gettimeofday(&end, NULL);
    seconds = (end.tv_sec - start.tv_sec);
    micros = ((seconds * 1000000) + end.tv_usec) - (start.tv_usec);

    double *A_from_GPU = (double*)malloc(sizeof(double));
    double *B_from_GPU = (double*)malloc(sizeof(double));
 
    // Copy data from device array d_C to host array C
    cudaMemcpy(A_from_GPU, d_A, sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(B_from_GPU, d_B, sizeof(double), cudaMemcpyDeviceToHost);
 
    printf("y= a + bx\n");
    printf("Shared Memory Linear Regression: A = %f and B = %f\n", *A_from_GPU, *B_from_GPU);
    printf("The elapsed time by GPU is %ds and %dus \n", seconds, micros); 


    free(X);
    free(Y);
    free(A_from_GPU);
    free(B_from_GPU);
 
    cudaFree(d_X);
    cudaFree(d_Y);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_sum_y);
    cudaFree(d_sum_x);
    cudaFree(d_sum_x_squared);
    cudaFree(d_sum_xy);

    return 0;
}

y= a + bx
Sequential Linear Regression: A = 4.499719 and B = 0.000016
The elapsed time by CPU is 2s and 1842252us 
y= a + bx
Shared Memory Linear Regression: A = -0.021426 and B = -10.501242
The elapsed time by GPU is 0s and 140269us 



## Global variable One Final Sum (GPU)


In [89]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <math.h>
#define N 50000000


__global__ void linear_regression_global(double *A, double *B, int *X, int *Y, int *sum_y, int *sum_x, int *sum_x_squared, int *sum_xy)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    int blockId = blockIdx.x;
    if(id < N) {
        atomicAdd(sum_y, Y[id]);
        atomicAdd(sum_x, X[id]);
        atomicAdd(sum_x_squared, X[id]*X[id]);
        atomicAdd(sum_xy, X[id] * Y[id]);
    }
 
    __syncthreads();
 
    if (id == 0) {
        *A = ((double) (*sum_y * *sum_x_squared) - (*sum_x * *sum_xy))/((N * *sum_x_squared) - pow(*sum_x, 2));
        *B = ((double) (N * *sum_xy) - (*sum_x * *sum_y))/((N * *sum_x_squared) - (*sum_x_squared));
    }

}

int main()
{
    // Number of bytes to allocate for N doubles
    size_t bytes = N*sizeof(int);

    // Allocate memory for arrays A, B on host
    int *X = (int*)malloc(bytes);
    int *Y = (int*)malloc(bytes);
    long double sum_y = 0.0;
    long double sum_x = 0.0;
    long double sum_x_squared = 0.0;
    long double sum_xy = 0.0;
    long double A = 0.0;
    long double B = 0.0;
  
    struct timeval start, end;

  
    for (int i = 0; i < N; i++) {
        X[i] = rand() % 10;
        Y[i] = rand() % 10;
    }

    gettimeofday(&start, NULL);

    
    for (int i = 0; i < N; i++) {
        sum_y += Y[i];
        sum_x += X[i];
        sum_x_squared += pow(X[i], 2);
        sum_xy += (X[i] * Y[i]);
    }
 
    // printf("sum_y %d\n", sum_y);
    // printf("sum_x %d\n", sum_x);
    // printf("sum_x_squared %llu\n", sum_x_squared);
    // printf("sum_xy %llu\n", sum_xy);
 
    A = ((sum_y * sum_x_squared) - (sum_x * sum_xy))/((N * sum_x_squared) - pow(sum_x, 2));
    B = ((N * sum_xy) - (sum_x * sum_y))/((N * sum_x_squared) - (sum_x_squared));
 
    gettimeofday(&end, NULL);
    long seconds = (end.tv_sec - start.tv_sec);
    long micros = ((seconds * 1000000) + end.tv_usec) - (start.tv_usec);
 
    printf("y= a + bx\n");
    printf("Sequential Linear Regression: A = %Lf and B = %Lf\n", A, B);
    printf("The elapsed time by CPU is %ds and %dus \n", seconds, micros); 
 
    int *d_X, *d_Y;
    double *d_A, *d_B;
    int *d_sum_y, *d_sum_x, *d_sum_x_squared, *d_sum_xy;
    cudaMalloc(&d_X, bytes);
    cudaMalloc(&d_Y, bytes);
    cudaMalloc(&d_A, sizeof(double));
    cudaMalloc(&d_B, sizeof(double));
    cudaMalloc(&d_sum_y, sizeof(int));
    cudaMalloc(&d_sum_x, sizeof(int));
    cudaMalloc(&d_sum_x_squared, sizeof(int)); 
    cudaMalloc(&d_sum_xy, sizeof(int)); 
 
    cudaMemcpy(d_X, X, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_Y, Y, bytes, cudaMemcpyHostToDevice);
 
    int thr_per_blk = 256;
    int blk_in_grid = ceil( float(N) / thr_per_blk );

    // Launch kernel
    gettimeofday(&start, NULL);
    linear_regression_global<<< blk_in_grid, thr_per_blk >>>(d_A, d_B, d_X, d_Y, d_sum_y, d_sum_x, d_sum_x_squared, d_sum_xy);
 
    cudaDeviceSynchronize();
 
    gettimeofday(&end, NULL);
    seconds = (end.tv_sec - start.tv_sec);
    micros = ((seconds * 1000000) + end.tv_usec) - (start.tv_usec);

    double *A_from_GPU = (double*)malloc(sizeof(double));
    double *B_from_GPU = (double*)malloc(sizeof(double));
 
    // Copy data from device array d_C to host array C
    cudaMemcpy(A_from_GPU, d_A, sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(B_from_GPU, d_B, sizeof(double), cudaMemcpyDeviceToHost);
 
    printf("y= a + bx\n");
    printf("Global Linear Regression: A = %f and B = %f\n", *A_from_GPU, *B_from_GPU);
    printf("The elapsed time by GPU is %ds and %dus \n", seconds, micros); 


    free(X);
    free(Y);
    free(A_from_GPU);
    free(B_from_GPU);
 
    cudaFree(d_X);
    cudaFree(d_Y);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_sum_y);
    cudaFree(d_sum_x);
    cudaFree(d_sum_x_squared);
    cudaFree(d_sum_xy);

    return 0;
}

y= a + bx
Sequential Linear Regression: A = 4.499719 and B = 0.000016
The elapsed time by CPU is 2s and 1995478us 
y= a + bx
Global Linear Regression: A = 0.073646 and B = -0.778793
The elapsed time by GPU is 0s and 5662us 

